In [1]:
import torch
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
import numpy as np
import cv2
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import os
import random
from tqdm import tqdm

In [2]:
# --- Configuration ---
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
OUTPUT_DIR = "attack_results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Target classes to attack (COCO labels: 1=person, 3=car, 13=stop sign)
TARGET_LABELS = [1, 3, 13]

In [3]:
def load_model():
    """Loads a pre-trained Faster R-CNN model."""
    print(f"Loading Faster R-CNN model on {DEVICE}...")
    model = fasterrcnn_resnet50_fpn(pretrained=True)
    model.to(DEVICE)
    model.eval() # Set to eval mode, but we will need gradients for the input
    return model

def get_sample_image():
    """Downloads a sample image for testing."""
    url = "https://raw.githubusercontent.com/pytorch/hub/master/images/dog.jpg"
    # Alternative complex scene:
    # url = "https://upload.wikimedia.org/wikipedia/commons/f/f0/Traffic_jam_in_Delhi.jpg"

    print(f"Downloading sample image from {url}...")
    response = requests.get(url)
    img = Image.open(BytesIO(response.content)).convert("RGB")
    return img

def preprocess(image):
    """Converts PIL image to Tensor and normalizes."""
    img_tensor = F.to_tensor(image).to(DEVICE)
    return img_tensor

def show_prediction(img_tensor, model, title, save_path=None, threshold=0.5):
    """Runs inference and visualizes predictions."""
    with torch.no_grad():
        predictions = model([img_tensor])[0]

    img_np = img_tensor.cpu().permute(1, 2, 0).numpy()
    img_np = np.clip(img_np, 0, 1)

    plt.figure(figsize=(10, 8))
    plt.imshow(img_np)

    ax = plt.gca()

    has_detection = False
    for box, label, score in zip(predictions['boxes'], predictions['labels'], predictions['scores']):
        if score > threshold:
            has_detection = True
            box = box.cpu().numpy()
            rect = plt.Rectangle((box[0], box[1]), box[2]-box[0], box[3]-box[1],
                                 fill=False, color='red', linewidth=2)
            ax.add_patch(rect)
            plt.text(box[0], box[1], f"{score:.2f}", color='red', fontsize=12, backgroundcolor='white')

    plt.title(title)
    plt.axis('off')

    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Saved result to {save_path}")

    plt.close()
    return has_detection

In [4]:
# --- 1. Digital Attack: PGD (Projected Gradient Descent) ---

def pgd_attack(model, img_tensor, epsilon=0.05, alpha=0.01, num_iter=20):
    """
    Performs a digital PGD attack.
    Goal: Perturb the WHOLE image slightly to suppress detections.
    """
    print("\n--- Starting Digital PGD Attack ---")

    # Clone image and enable gradients
    adv_img = img_tensor.clone().detach()
    adv_img.requires_grad = True

    optimizer = torch.optim.SGD([adv_img], lr=alpha)

    for i in tqdm(range(num_iter)):
        # Forward pass
        # Faster R-CNN returns a loss dictionary during training mode.
        # However, we are attacking the inference output.
        # Standard trick: We need to maximize the loss of the correct detections
        # OR minimize the confidence of the detections.

        # For simplicity in this prototype, we force the model to 'train' mode
        # momentarily to get loss values for the original ground truth (or pseudo-ground truth).

        # 1. Get pseudo-ground truth from clean image
        model.eval()
        with torch.no_grad():
            clean_preds = model([img_tensor])[0]

        # Filter only high confidence targets to suppress
        targets = {
            'boxes': clean_preds['boxes'][clean_preds['scores'] > 0.5],
            'labels': clean_preds['labels'][clean_preds['scores'] > 0.5]
        }

        if len(targets['boxes']) == 0:
            print("No objects to attack.")
            break

        # 2. Switch to train mode to compute loss gradients
        model.train()

        # We want to MAXIMIZE this loss (make the model confused)
        # FasterRCNN computes loss automatically if targets are provided
        loss_dict = model([adv_img], [targets])
        total_loss = sum(loss for loss in loss_dict.values())

        # Zero gradients
        optimizer.zero_grad()

        # Backward pass
        total_loss.backward()

        # Update image: We want to INCREASE loss, so we ADD gradient
        data_grad = adv_img.grad.data
        adv_img.data = adv_img.data + alpha * data_grad.sign()

        # Projection (Clip noise to epsilon ball)
        eta = torch.clamp(adv_img.data - img_tensor.data, -epsilon, epsilon)
        adv_img.data = torch.clamp(img_tensor.data + eta, 0, 1)

        # Reset gradient for next step
        adv_img.grad.data.zero_()

    model.eval()
    return adv_img.detach()

In [5]:
# --- 2. Physical Attack Simulation: Adversarial Patch with EOT ---

class AdversarialPatch:
    def __init__(self, patch_shape=(3, 100, 100)):
        """
        patch_shape: (Channels, Height, Width)
        """
        # Initialize patch with random noise or gray
        self.patch = torch.rand(patch_shape, device=DEVICE, requires_grad=True)
        self.patch_shape = patch_shape

    def apply_patch(self, img_tensor, patch, location=None):
        """
        Applies the patch to the image.
        If location is None, places it randomly (simulating EOT).
        """
        _, h, w = img_tensor.shape
        ph, pw = self.patch_shape[1], self.patch_shape[2]

        patched_img = img_tensor.clone()

        if location is None:
            # Random location (EOT aspect)
            x = random.randint(0, w - pw)
            y = random.randint(0, h - ph)
        else:
            x, y = location

        # Overlay patch
        # In a real physical simulator, we would use affine transforms (rotation/perspective) here.
        # For this prototype, we do a simple overlay + noise to simulate print imperfections.

        # Simulating "Physical" noise (printing errors, lighting)
        noise = torch.randn_like(patch) * 0.05
        robust_patch = torch.clamp(patch + noise, 0, 1)

        patched_img[:, y:y+ph, x:x+pw] = robust_patch
        return patched_img

    def optimize(self, model, ref_img_tensor, epochs=50, lr=0.05):
        print("\n--- Starting Physical Patch Optimization (EOT) ---")

        # We optimize ONLY the patch
        optimizer = torch.optim.Adam([self.patch], lr=lr)

        # Get pseudo-labels to attack
        model.eval()
        with torch.no_grad():
            initial_preds = model([ref_img_tensor])[0]

        # If no objects found, nothing to attack
        if len(initial_preds['boxes']) == 0:
            print("No objects found to attach patch to.")
            return

        targets = {
            'boxes': initial_preds['boxes'],
            'labels': initial_preds['labels']
        }

        for epoch in tqdm(range(epochs)):
            # Expectation Over Transformation (EOT) Loop
            # We average gradients over multiple random transformations/placements

            acc_loss = 0
            optimizer.zero_grad()

            # Mini-batch of transformations (e.g., 5 random placements per step)
            for _ in range(5):
                # Apply patch at random location (Transformation)
                patched_img = self.apply_patch(ref_img_tensor, self.patch, location=None)

                # We need to maximize object loss (make objects disappear)
                model.train()
                loss_dict = model([patched_img], [targets])

                # Losses: classifier loss + box regression loss
                # Maximizing these makes the model fail to detect the ground truth
                loss = sum(loss for loss in loss_dict.values())

                # Backward
                # We want to MAXIMIZE loss, so we minimize -loss
                (-loss).backward()
                acc_loss += loss.item()

            # Update patch
            optimizer.step()

            # Clamp patch to be a valid image (0-1) - "Printability" constraint
            self.patch.data.clamp_(0, 1)

        model.eval()
        print("Patch optimization complete.")

In [7]:
# --- Main Execution ---

def main():
    # 1. Setup
    model = load_model()
    img_pil = get_sample_image()
    img_tensor = preprocess(img_pil)

    # 2. Baseline
    print("Running baseline inference...")
    show_prediction(img_tensor, model, "Original Image",
                   save_path=os.path.join(OUTPUT_DIR, "1_baseline.png"))

    # 3. Digital Attack (PGD)
    adv_tensor_pgd = pgd_attack(model, img_tensor, epsilon=0.1, num_iter=30)
    print("Running inference on Digital Attack...")
    show_prediction(adv_tensor_pgd, model, "Digital PGD Attack",
                   save_path=os.path.join(OUTPUT_DIR, "2_digital_attack.png"))

    # Save the noise itself for visualization
    noise = (adv_tensor_pgd - img_tensor).abs().cpu().permute(1, 2, 0).numpy() # Amplify for visibility
    plt.imsave(os.path.join(OUTPUT_DIR, "2b_digital_noise_pattern.png"), np.clip(noise * 10, 0, 1))

    # 4. Physical Attack (Adversarial Patch)
    # Create a patch roughly 15% of image size
    h, w = img_tensor.shape[1], img_tensor.shape[2]
    patch_size = int(min(h, w) * 0.2)
    attacker = AdversarialPatch(patch_shape=(3, patch_size, patch_size))

    # Train the patch
    attacker.optimize(model, img_tensor, epochs=20)

    # Apply optimized patch to a specific location for final visualization
    # (e.g., center of image)
    final_patch_img = attacker.apply_patch(img_tensor, attacker.patch, location=(w//2 - patch_size//2, h//2 - patch_size//2))

    print("Running inference on Physical Patch Attack...")
    show_prediction(final_patch_img.detach(), model, "Physical Patch Attack",
                   save_path=os.path.join(OUTPUT_DIR, "3_physical_attack.png"))

    # Save the isolated patch
    patch_np = attacker.patch.detach().cpu().permute(1, 2, 0).numpy()
    plt.imsave(os.path.join(OUTPUT_DIR, "3b_generated_patch.png"), patch_np)

    print("\nDone! Check the 'attack_results' folder.")

if __name__ == "__main__":
    main()

Loading Faster R-CNN model on cuda...


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Running baseline inference...
Saved result to attack_results/1_baseline.png

--- Starting Digital PGD Attack ---


100%|██████████| 30/30 [00:10<00:00,  2.98it/s]


Running inference on Digital Attack...
Saved result to attack_results/2_digital_attack.png

--- Starting Physical Patch Optimization (EOT) ---


100%|██████████| 20/20 [00:24<00:00,  1.21s/it]


Patch optimization complete.
Running inference on Physical Patch Attack...
Saved result to attack_results/3_physical_attack.png

Done! Check the 'attack_results' folder.
